<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px" />

# Excel Madness Lab!

_Author:_ Tim Book

## Our Mission
We work for a large supermarket chain, with stores in 10 major cities that happen to coincide with General Assembly campuses. However, this company's idea of a "database" is just a bunch of Excel spreadsheets! In order to analyze our data, we're going to need to process the existing data into a form we can use. **Our end goal is to have one csv per city.**

## Cleanup Duty!
It is a hard truth that data scientists spend a large majority of their time cleaning data. Data never arrives on our desks in exactly the format in which we want it, and it's up to us to transform it to a workable format.

Being good cleaning, moving, and reshaping data is in itself a valuable and employable job skill. If you follow these directions exactly, we will walk through constructing an automated process for processing data from this supermarket chain.

# Part I: Processing

### Step 1: Imports and the `os` library
We're going to import three libraries: numpy, pandas, and `os`.

In [1]:
# Import libraries here.
import numpy as np
import pandas as pd
import os

The `os` library is extremely useful for performing system commands from within Python. Let's get two pieces of overhead out of the way now:

1. Create an `output` folder using `os.mkdir()`
2. Create a variable called `files` that is the list of files in the `data` folder using `os.listdir()`

**WARNING:** The `os.mkdir()` function will give you an error if you try to make a folder that already exists!

In [2]:
# Create an output folder.
#os.mkdir('output')
# Create a files variable that contains all of our data files.
files = os.listdir('data')

In [6]:
files.remove('.ipynb_checkpoints')

In [7]:
files

['Jan 23.xlsx',
 'Jan 19.xlsx',
 'Jan 15.xlsx',
 'Jan 7.xlsx',
 'Jan 6.xlsx',
 'Jan 14.xlsx',
 'Jan 18.xlsx',
 'Jan 22.xlsx',
 'Jan 29.xlsx',
 'Jan 13.xlsx',
 'Jan 1.xlsx',
 'Jan 25.xlsx',
 'Jan 24.xlsx',
 'Jan 12.xlsx',
 'Jan 28.xlsx',
 'Jan 3.xlsx',
 'Jan 11.xlsx',
 'Jan 27.xlsx',
 'Jan 31.xlsx',
 'Jan 30.xlsx',
 'Jan 26.xlsx',
 'Jan 10.xlsx',
 'Jan 2.xlsx',
 'Jan 9.xlsx',
 'Jan 21.xlsx',
 'Jan 5.xlsx',
 'Jan 17.xlsx',
 'Jan 16.xlsx',
 'Jan 4.xlsx',
 'Jan 20.xlsx',
 'Jan 8.xlsx']

### Step 2: Process one data frame
It looks like we have data for the month of January. 31 files of 10 sheets each! Luckily they are all in the same format. So let's read just one in and process that. It might be helpful to open one up in your spreadsheet viewer of choice first (Excel, Numbers, Sheets, etc.)

In [8]:
# Read in data from your city from January 1st.
jan1 = pd.ExcelFile('./data/Jan 1.xlsx')
newyork_jan1=pd.read_excel(jan1,'New York')
newyork_jan1

,prodcode,price_eu,weight_kg,quantity
0,4159,2.048141,8.812961,138
1,4062,2.728485,9.331372,184
2,4404,2.182498,8.937798,491
3,4412,2.342694,7.067682,218
4,4131,1.802839,9.551503,391
...,...,...,...,...
124,4796,1.827676,7.922745,444
125,4225,2.463229,5.839095,258
126,4134,0.920152,9.225959,395
127,3302,1.510998,2.667668,168


In [ ]:
# boston=pd.read_excel(jan1,'Boston')
# boston.head()

### Step 2a: Convert to 'Merican columns
For whatever reason, our data are stored in euros and kilograms. Create `price_usd` and `weight_lb` columns. There are 2.2 pounds per kilogram, and 1.1 dollars per euro.

In [9]:
newyork_jan1['price_usd']=newyork_jan1['price_eu']*1.1

In [10]:
newyork_jan1['weight_lb']=newyork_jan1['weight_kg']*2.2

In [11]:
newyork_jan1.head()

,prodcode,price_eu,weight_kg,quantity,price_usd,weight_lb
0,4159,2.048141,8.812961,138,2.252955,19.388515
1,4062,2.728485,9.331372,184,3.001334,20.529018
2,4404,2.182498,8.937798,491,2.400748,19.663156
3,4412,2.342694,7.067682,218,2.576964,15.548899
4,4131,1.802839,9.551503,391,1.983123,21.013306


### Step 2b: Merge in product names
You'll notice we also have a `plu-codes.csv` file containing actual product names matched up against their price lookup (PLU) codes. Let's merge these product names onto our Jan 1 data.
* _Hint 1:_ What kind of merge is this? Right, left, inner, outer, etc.?
* _Hint 2:_ Pay special attention to column names!

In [12]:
plu = pd.read_csv("plu-codes.csv")
plu.head()

,product,plu_code
0,Apple (Fuji),4131
1,Apple (Gala),4134
2,Apricot,3302
3,Avocado,4225
4,Banana,4011


In [13]:
newyork_jan1.rename(columns={'prodcode':'plu_code'},inplace=True)
newyork_jan1.head()

,plu_code,price_eu,weight_kg,quantity,price_usd,weight_lb
0,4159,2.048141,8.812961,138,2.252955,19.388515
1,4062,2.728485,9.331372,184,3.001334,20.529018
2,4404,2.182498,8.937798,491,2.400748,19.663156
3,4412,2.342694,7.067682,218,2.576964,15.548899
4,4131,1.802839,9.551503,391,1.983123,21.013306


In [14]:
newyork_jan1=newyork_jan1.merge(plu, how='left')

### Step 2c: Drop unnecessary columns
We've created some extraneous columns. Drop the old price and weight columns, as well as any redundant columns.

In [15]:
newyork_jan1.head()

,plu_code,price_eu,weight_kg,quantity,price_usd,weight_lb,product
0,4159,2.048141,8.812961,138,2.252955,19.388515,Onion
1,4062,2.728485,9.331372,184,3.001334,20.529018,Cucumber
2,4404,2.182498,8.937798,491,2.400748,19.663156,Peach
3,4412,2.342694,7.067682,218,2.576964,15.548899,Pear
4,4131,1.802839,9.551503,391,1.983123,21.013306,Apple (Fuji)


In [16]:
newyork_jan1=newyork_jan1.drop(['price_eu','weight_kg'],axis=1)

### Step 2d: Add the date
Simply create a new `date` column that is the date this data was collected. For example, if this is from `Jan 1.xlsx`, this column should be full of `Jan 1`.

In [17]:
newyork_jan1['date']='jan1'

In [18]:
newyork_jan1.head()

,plu_code,quantity,price_usd,weight_lb,product,date
0,4159,138,2.252955,19.388515,Onion,jan1
1,4062,184,3.001334,20.529018,Cucumber,jan1
2,4404,491,2.400748,19.663156,Peach,jan1
3,4412,218,2.576964,15.548899,Pear,jan1
4,4131,391,1.983123,21.013306,Apple (Fuji),jan1


### Step 3: Write a function that conducts all of Step 2
This function should import a **filename and a city name** and return a fully processed DataFrame. That is, the function should:
1. Read in the data from the given file and city.
1. Create USD and pound columns.
1. Merge in product names.
1. Drop unnecessary columns.
1. Add a date column

In [19]:
def process_data(filename, city):
    finaldate=str(filename).replace('.xlsx','')
    filename=pd.ExcelFile('./data/'+filename) #opening excel
    df=pd.read_excel(filename, city) #selecting tab within the excel based on city name + creating dataframe
    df['price_usd']=df['price_eu']*1.1 #adding USD
    df['weight_lb']=df['weight_kg']*2.2 #adding LB
    df.rename(columns={'prodcode':'plu_code'},inplace=True) #renaming column to merge
    df=df.merge(plu, how='left') #merging
    df=df.drop(['price_eu','weight_kg'],axis=1)#dropping erroneous rows
    df['date']=(finaldate) #adding date column
    return df 


In [20]:
filename = 'Jan 10.xlsx'
city = 'New York'
process_data('Jan 10.xlsx','New York')

,plu_code,quantity,price_usd,weight_lb,product,date
0,4272,319,2.991243,16.377081,Grapes,Jan 10
1,4062,485,1.705711,4.733293,Cucumber,Jan 10
2,4404,288,0.690103,9.681877,Peach,Jan 10
3,4272,459,1.515029,19.414309,Grapes,Jan 10
4,4225,449,2.811033,21.446132,Avocado,Jan 10
...,...,...,...,...,...,...
166,4958,257,2.971513,12.062531,Lemon,Jan 10
167,4012,402,0.937014,3.482773,Orange,Jan 10
168,4650,227,2.236302,19.022781,Mushroom,Jan 10
169,4012,234,1.576208,15.479361,Orange,Jan 10


Test your function out on a new file and city!

In [21]:
process_data('Jan 10.xlsx', 'Los Angeles')

,plu_code,quantity,price_usd,weight_lb,product,date
0,4272,374,3.215805,18.458957,Grapes,Jan 10
1,4134,386,1.101397,4.144168,Apple (Gala),Jan 10
2,4650,254,2.462905,16.272903,Mushroom,Jan 10
3,4309,220,1.402996,13.852859,Lychee,Jan 10
4,4012,326,2.383897,10.723717,Orange,Jan 10
...,...,...,...,...,...,...
194,4650,213,1.305891,5.691630,Mushroom,Jan 10
195,4796,290,1.865070,19.355372,Tomato,Jan 10
196,4012,448,3.055497,7.577243,Orange,Jan 10
197,4030,391,1.092079,19.929001,Kiwi,Jan 10


### Step 4: Process all of January's data
For each spreadsheet, process the data and store the resulting DataFrame in one big list. **You only need to do this for your city!**

* _Hint 1:_ A listcomp would make this whole step one line of code!
* _Hint 2:_ You've already made that `files` variable to help you here.

In [49]:
#listcomp with help from Gaby Burgos
dataframe=[process_data(i,'San Francisco') for i in files]

### Step 5: Concatenate all DataFrames from Step 4 into one large DataFrame
* _Hint:_ Is there a function in `pandas` that can do something like this for us? This is also just one line of code!

In [ ]:
New York
Boston
Washington, DC
Atlanta
Chicago
Denver
Austin
Seattle
Los Angeles
SanFrancisco

In [50]:
df_SanFrancisco=pd.concat(dataframe)
df_SanFrancisco.head(1)

,plu_code,quantity,price_usd,weight_lb,product,date
0,4958,197,1.946762,11.36345,Lemon,Jan 23


In [51]:
df_LosAngeles.sort_values(by='date', ascending=True)

,plu_code,quantity,price_usd,weight_lb,product,date
70,4011,448,2.991470,13.003488,Banana,Jan 1
41,4404,497,0.924926,15.063898,Peach,Jan 1
71,4081,278,1.970743,15.540793,Eggplant,Jan 1
72,4159,115,1.718897,7.670208,Onion,Jan 1
73,4240,264,1.361914,11.960528,Blueberries,Jan 1
...,...,...,...,...,...,...
48,4404,393,1.859187,6.568451,Peach,Jan 9
49,4078,317,2.667788,9.636454,Corn,Jan 9
50,4012,168,2.520105,2.399348,Orange,Jan 9
34,4078,381,0.615152,4.463558,Corn,Jan 9


### Step 6: Do this for all cities, write data
Here's the big one. For each city, process and the data as in steps 3-5, and then write the data to our `output` folder. Below is a dictionary of city name to desired output file name.

Before writing your DataFrame, do the following:
* Add a `city` column
* Reorder the columns into the following order:


| city | date | product | prodcode | quantity | weight_lb | price_usd |
|---|---|---|---|---|---|---|

* _Hint:_ You can reorder DataFrame columns simply by writing over your DataFrame with itself, but specifying the specific column order with `.loc`. For example:

`print(df)`

| b | c | a |
|---|---|---|
| 1 | 2 | 3 |

`df = df.loc[:, ["a", "b", "c"]]`

`print(df)`

| a | b | c |
|---|---|---|
| 3 | 1 | 2 |


In [ ]:
df_newyork
df_boston
df_wdc
df_atlanta
df_Chicago
df_Denver
df_Austin
df_Seattle
df_LosAngeles

In [115]:
df_LosAngeles['city']=('Los Angeles')

In [116]:
df_LosAngeles=df_LosAngeles.loc[:,['city','date','product','plu_code','quantity','weight_lb','price_usd']]
df_LosAngeles.head(1)

,city,date,product,plu_code,quantity,weight_lb,price_usd
0,Los Angeles,Jan 23,Lemon,4958,263,12.163276,2.880092


In [ ]:
city_dict = {
    "Atlanta": "atl.csv",
    "Austin": "atx.csv",
    "Boston": "bos.csv",
    "Chicago": "chi.csv",
    "Denver": "den.csv",
    "Los Angeles": "lax.csv",
    "New York": "nyc.csv",
    "San Francisco": "sf.csv",
    "Seattle": "sea.csv",
    "Washington, DC": "dc.csv"
}

In [122]:
df_LosAngeles.to_csv('/2.02-lab-excel-madness-master/output/')

FileNotFoundError: [Errno 2] No such file or directory: '/2.02-lab-excel-madness-master/output/'

In [ ]:
# Loop through city_dict and carry out Step 6 here.

# The keys of city_dict can serve as the sheet name.
# The values of city_dict are what you should name the output .csv files.
# If done correctly, this cell could take almost a minute to run!

# Part II: Checking our answers 
In steps very similar to the ones conducted above...
1. Loop through the files we just wrote to `output`, and read them in, collecting them all in one list
1. Combine all of those DataFrames into one large DataFrame
1. For each city, find the mean `quantity`, `weight_lb`, and `price_usd`.

If you've done everything correct, your answer should look exactly like this:

![](imgs/correct-output.png)

# Part III (BONUS): Get this process production-ready!
_This part of the lab is optional, but very highly recommended, as the skills developed in this part are extremely common in industry._

For this step, we're going to take this whole process and put it into a production-ready Python script. **ABSOLUTELY NONE OF THIS STEP SHOULD TAKE PLACE IN A JUPYTER NOTEBOOK! PRODUCTIONALIZED ETL (_"Extract, Transform, Load"_) CODE DOES NOT TAKE PLACE IN A JUPYTER NOTEBOOK!**

The instructions are simple: As conducted in this lab, read, transform, and export the data in our Excel files into .csv files. The code should be in a `.py` file and executable from the command line. Here are some hints and tips to guide you:

### Hints, tips, and tricks:
* A good place to start is with the code you've already written. In this notebook, you can click `File > Download as > Python (.py)` to export as a `.py` file. Most of this exercise then comes down to you cleaning this file. (There will be a lot to clean).
* Remember `os.mkdir()` will throw an error if the folder you're trying to make already exists. Maybe you should check to see if it already exists. If it already exists, what should you do? (Remember that `.csv` can be overwritten with no problem.) The functions that can help you with this are all in the `os` library.
* Remember to follow all of the Python best practices we've already learned:
    - All import statements should go at the top of your script.
    - Comment your code. Comments shouldn't explain _what_ code does, but _why_ the code does this.
    - Keep your code DRY (don't repeat yourself) as opposed to WET (write everything twice). All constants should be variables that only need to be changed once. All code should be bottled into functions so you only need to fix it once.
* Make sure not to hardcode "Jan" anywhere. The point is that this code can be run throughout the lifetime of this supermarket's project, which is likely months or years. Keep your code so that if you get February data, you only need to change one tiny piece of the script (probably a file path).